In [1]:
// 次の３つの値をご自身のものに置き換えてください。Replace the following three values with your resources. 
const string deployment = "your-aoai-deployment-name";
const string apikey = "your-apai-api-key";
const string baseUrl = "https://{your-aoai-resource-name}.openai.azure.com";

In [39]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.6"
#r "nuget: Azure.Identity"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.6 Azure.Identity, 1.9.0

In [21]:
using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using System.Text.Json;

In [31]:
List<FunctionDefinition> functions =  new() {
    new()
    {
        Name = "get_recipe",
        Description = "与えられた料理のレシピを取得します。",
        Parameters = BinaryData.FromObjectAsJson(new
        {
            Type = "object",
            Properties = new
            {
                Food = new
                {
                    Type = "string",
                    Description = "レシピを検索する料理、例：ポトフ、ピラフ",
                },
            },
            Required = new[] { "food" },
        },
        new JsonSerializerOptions
        {
            PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
        }),
    }
};

In [15]:
var client = new  OpenAIClient(
    new Uri(baseUrl),
    new AzureKeyCredential(apikey));

In [32]:
var response = await client.GetChatCompletionsAsync(deployment,
    new ChatCompletionsOptions
    {
        Messages =
        {
            new() { Role = ChatRole.System, Content = "あなたはさまざまな事をサポートするアシスタントです。" },
            new() { Role = ChatRole.User, Content = "今日の夕食は外食にしようと思ったけど、大根が余っているからぶり大根を作ろうかな。" },
        },
        Functions = functions
    });

In [38]:
var finisReason = response.Value.Choices[0].FinishReason;

if (finisReason == "function_call"){
    var functionCall = response.Value.Choices[0].Message.FunctionCall;
    Console.WriteLine(functionCall.Name);
    Console.WriteLine(functionCall.Arguments);
}

get_recipe
{
  "food": "ぶり大根"
}
